<h1>Video Sliding Windows</h1>

<p>
So far we restricted ourselves to 1D time series, but the idea of recovering periodic dynamics with geometry can just as easily apply to multivariate signals.  In this module, we will examine sliding windows of videos as an exmaple.  Many natural videos also have periodicity, such as this video of a woman doing jumping jacks
</p>

[Jumping jacks video]

<p>
Video can be decomposed into a 3D array, which has dimensions width x height x time.  To tease out periodicity in geometric form, we will do the exact same thing as with sliding window 1D signal embeddings, but instead of just one sample per time shift, we need to take every pixel in every frame in the time window.  The figure below depicts this
</p>

<img src = "VideoStackTime.svg"><BR><BR>

To see this visually in the video next to PCA of the embedding, look at the following video

[Jumping jacks sliding window PCA video]

*Make a remark about practical issues with delay embeddings of videos; need PCA


In [ ]:
#Given that the period is 30 frames per cycle, choose a dimension and a Tau that capture 
#this motion in the roundest possible way
#Plot persistence diagram and PCA